In [ ]:

!pip install simpletransformers


## 1. Import the data:
## Note: replace variable 'path' to dataset :  [ner_dataset.csv](https://github.com/sriramrokkam/NLP_NER_BERT/blob/BITS/ner_dataset.csv)

In [ ]:
import pandas as pd
path = "/content/ner_dataset.csv"
data = pd.read_csv(path,encoding="latin1" )
data.head(30)
data = data.head(500) ## change value for number of records.

## 2. Fill the missing values

In [ ]:
data =data.fillna(method ="ffill")
data.head(30)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )
data.head(30)

In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

## 3. Split Data: Training and Testing

In [ ]:
data["labels"] = data["labels"].str.upper()

X= data[["sentence_id","words"]]
Y =data["labels"]

x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})
train_data

## 4. Build and Train Model

In [ ]:
from simpletransformers.ner import NERModel,NERArgs
label = data["labels"].unique().tolist()
label

In [ ]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [ ]:
model = NERModel('bert', 'bert-base-cased', labels=label, args=args, use_cuda=False)

In [ ]:
r = model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

In [ ]:
result

## 5. Evaluation

In [ ]:
validation_sentence = ["They marched from the Houses of Parliament to a rally in Hyde Park ."]
prediction, model_output = model.predict(validation_sentence)

In [ ]:
prediction

## 6. Save Tokenizer and Model

In [ ]:
# Save the model and tokenizer after training
# model.save_model("ner_model_v1")
import shutil
import os

# Define the model folder path
model_dir = "ner_model"

# Check if the folder exists and remove it if necessary
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)  # This will delete the existing folder and its contents

# Save the model and tokenizer after deleting the existing folder
model.save_model(model_dir)


## 7. Load the Model
## Note: 1.
1.   use_cuda = True (for GPU) and False (for CPU)
2.   Check the 'outputs' folder.
##       

In [ ]:
from simpletransformers.ner import NERModel

# Load the saved model
# Add the 'args' parameter to specify the configuration
model = NERModel(
    "bert",
    "outputs",
    use_cuda=False,
    args={"overwrite_output_dir": True} # This allows you to overwrite the output directory if it already exists
)


# 8. Compare the Results

In [ ]:
#
predictions, raw_outputs = model.predict(test_Sentence)

# Print the predictions
print(predictions)
